In [2]:
# SBS 뉴스 Crawling

from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pandas as pd

from time import sleep

In [2]:
url = "https://news.sbs.co.kr"
url_main = "https://news.sbs.co.kr/news/newsflash.do?pageDate=20210920"
url_target = "https://news.sbs.co.kr/news/newsflash.do?pageDate=20210920"

In [3]:
columns = ['title', 'headline', 'date', 'news_link', 'content', 'category']
data = []

In [ ]:
print('----- Start -----')

while True:
    current_page = requests.get(url=url_main) # main page(idx=1)
    while True:
        if current_page.status_code == 200:
            current_page = current_page.text
            break
        else:
            print(current_page.status_code)
            
    # sleep(5)
    soup = bs(current_page, 'html.parser')
    
    link = soup.find('link', {'rel' : 'canonical'})['href']
    date = link.split('?pageDate=')[-1]
    if date == '20220920':
        break
    
    next_date = list(soup.find('ol',  {'class' : 'mdp_date_list'})) # ['\n', date]
    next_date = [e for e in next_date if e != '\n'][1] # remove '\n'
    next_url = next_date.find('a')['href']
    
    idx_p = 1

    print()
    print('date: ' + date)
    print()
    
    while True:
        print()
        print('pageIdx=' + str(idx_p))
        print()
        
        current_page = requests.get(url=url_target) # next page(idx > 1)
        while True:
            if current_page.status_code == 200:
                current_page = current_page.text
                break
            else:
                print(current_page.status_code)
            
        # sleep(5)
        soup = bs(current_page, 'html.parser')
        
        flag = soup.find('div', {'class' : 'w_news_list type_issue'}).text.strip() # content presence
        if flag == '해당 일자의 뉴스 정보가 없습니다.':
            break
        
        list_news = list(soup.find_all('li', {'itemtype' : 'http://schema.org/NewsArticle'})) # current page news
        
        for news in list_news:
            try:
                title = news.find('strong').text # title
                headline = news.find('span', {'class' : 'read'}).text # headline
                # thumb = news.find('img')['src'] # thumbnail
                date = news.find('span', {'class' : 'date'}).text # date
                link_news = news.find('meta')['itemid'] # news link
            except (AttributeError, TypeError):
                title = None
                headline = None
                # thumb = None
                date = None
                link_news = None
            
            article_page = requests.get(url=link_news) # news page
            while True:
                if article_page.status_code == 200:
                    article_page = article_page.text
                    break
                else:
                    print(article_page.status_code)
                    
            # sleep(5) 
            soup2 = bs(article_page, 'html.parser')
            
            try:
                content = soup2.find('div', {'itemprop' : 'articleBody'}).text.replace('\n', '') # content
                category = soup2.find('li', {'class' : 'cate03'}).text # category
            except (AttributeError, TypeError):
                content = None
                category = None

            if title != None and headline != None and date != None and link_news != None and content != None and category != None:
                data.append([title, headline, date, link_news, content, category])
            
            print(len(data))
        
        idx_p += 1
        page_idx = '&pageIdx=' + str(idx_p)
        url_target = url_main + page_idx
            
    url_main = url + next_url
    url_target = url + next_url

print()
print('----- End -----')

In [9]:
dataset = pd.DataFrame(data=data, columns=columns)
dataset['site'] = 'sbs'
dataset.to_csv('./Data/kbs.csv', index=False)

In [ ]:
# span class read : 헤드라인 ~
# span class thumb : 썸네일 ~
# strong class sub : 뉴스 제목 ~
# span class data : 뉴스 날짜 및 시간 ~ 
# meta itemid : 뉴스 링크 ~


# li class cate03 : 카테고리 ~
# div class itemprop articleBody : 뉴스 내용

# div class mdp_inner -> a href #page : 
# div class w_news_list type_issue : 뉴스 정보